### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The TuringModels pkg imports modules such as CSV and DataFrames

In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/dev/Turing/src/Turing.jl:24


Read in the `rugged` data as a DataFrame

In [2]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');

Show size of the DataFrame (should be 234x51)

In [3]:
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [4]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [5]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [6]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size (should equal 170 x 52)

In [7]:
size(dd)

(170, 52)

Define the Turing model

In [8]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

Use Turing mcmc

In [9]:
posterior = sample(m8_1stan(dd[:log_gdp], dd[:rugged], dd[:cont_africa]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.05
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.026792728411377444, std = [1.0, 1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 252.91359624099962;
  #lf / sample        = 0.0;
  #evals / sample     = 46.143;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].


Describe the posterior samples

In [10]:
describe(posterior)

Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean           SD         Naive SE       MCSE         ESS   
       α    9.177727030  0.544957413 0.01218561820 0.0387707887  197.56766
  lf_num    0.000000000  0.000000000 0.00000000000 0.0000000000        NaN
      βA   -1.904083647  0.436209405 0.00975393883 0.0357953175  148.50382
      βR   -0.187853634  0.178975354 0.00400201057 0.0123477823  210.09152
       σ    0.971095210  0.272548554 0.00609437094 0.0204594203  177.46026
 elapsed    0.126456798  0.101997799 0.00228074011 0.0046814513  474.70147
 epsilon    0.028589068  0.018914646 0.00042294434 0.0006114057  957.05620
eval_num   46.143000000 26.523749841 0.59308907664 0.5981432235 1966.34389
     βAR    0.380371851  0.177224637 0.00396286336 0.0098604808  323.03681
      lp -250.206770871 19.995784933 0.44711934374 1.6466746899  147.45561
  lf_eps    0.028589068  0.018914646 0.00042294434 0.0006114057  

Fix the inclusion of adaptation samples

In [11]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names)

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Union{Missing, Float64}[9.14611 0.0 … -246.759 0.0267927; 9.19438 0.0 … -253.237 0.0267927; … ; 9.41211 0.0 … -248.038 0.0267927; 9.11704 0.0 … -248.546 0.0267927]

Example of a Turing run simulation output

Here's the ulam() output from rethinking

In [12]:
m8_1s_cmdstan = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
    a  9.22360053 0.139119116 0.0021996664 0.0034632816 1000
   bR -0.20196346 0.076106388 0.0012033477 0.0018370185 1000
   bA -1.94430980 0.227080488 0.0035904578 0.0057840746 1000
  bAR  0.39071684 0.131889143 0.0020853505 0.0032749642 1000
sigma  0.95036370 0.052161768 0.0008247500 0.0009204073 1000

Quantiles:
          2.5%       25.0%       50.0%      75.0%        97.5%
    a  8.95307475  9.12719750  9.2237750  9.31974000  9.490234250
   bR -0.35217930 -0.25334425 -0.2012855 -0.15124725 -0.054216855
   bA -2.39010825 -2.09894500 -1.9432550 -1.78643000 -1.513974250
  bAR  0.13496995  0.30095575  0.3916590  0.47887625  0.650244475
sigma  0.85376115  0.91363250  0.9484920  0.98405750  1.058573750
";

Describe the posterior samples

In [13]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                 SD                       Naive SE                      MCSE                ESS   
       α    9.21318649  0.14668651825035525226859 0.0046386349971097966260469 0.0117056981694366648433236  157.03113
  lf_num    0.00000000  0.00000000000000000000000 0.0000000000000000000000000 0.0000000000000000000000000        NaN
      βA   -1.93095798  0.24179755964384133659628 0.0076463102114495071215394 0.0188531924995661466504249  164.48793
      βR   -0.19767909  0.08310633728449441648412 0.0026280531381317512491791 0.0058432367373252970879638  202.28390
       σ    0.95237692  0.05361465844528068985797 0.0016954443665906904735452 0.0017804139850477013976709  906.82836
 elapsed    0.13435410  0.12416787454202164731054 0.0039265329577482513054298 0.0083376417742839804836397  221.78493
 epsilon    0.02679273  0.00000000000000003471183 0.0000000000000000010

end of 08/m8.1t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*